# Bind runtime args(运行时参数绑定)
有时我们想要在一个Runnable序列中调用一个带有常量参数的Runnable，这些参数不是前面Runnable输出的一部分，也不是用户输入的一部分。我们可以使用Runnable.bind()来轻松地传递这些参数。

假设我们有一个简单的提示+模型序列：

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 创建AI模型
model = ChatOpenAI(
)

In [10]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it. Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
        ),
        ("human", "{equation_statement}"),
    ]
)

runnable = (
    {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()
)

print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12

SOLUTION: 
Subtract 7 from both sides:
x^3 = 5

Take the cube root of both sides:
x = ∛5


并且想要使用特定的停用词调用模型：

In [19]:
runnable = (
    {"equation_statement": RunnablePassthrough()}
    | prompt
    | model.bind(stop="SOLUTION")
    | StrOutputParser()
)

In [21]:
print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12



### Attaching OpenAI functions（添加OpenAI运行函数）

绑定的一个特别有用的应用是将OpenAI函数附加到兼容的OpenAI模型上

In [22]:
function = {
    "name": "solver",
    "description": "Formulates and solves an equation",
    "parameters": {
        "type": "object",
        "properties": {
            "equation": {
                "type": "string",
                "description": "The algebraic expression of the equation",
            },
            "solution": {
                "type": "string",
                "description": "The solution to the equation",
            },
        },
        "required": ["equation", "solution"],
    },
}

In [23]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it.",
        ),
        ("human", "{equation_statement}"),
    ]
)

In [26]:
model = ChatOpenAI(
  temperature=0,
).bind(
    function_call={"name": "solver"}, functions=[function]
)

In [27]:
runnable = {"equation_statement": RunnablePassthrough()} | prompt | model
runnable.invoke("x raised to the third plus seven equals 12")

AIMessage(content='The equation is: x^3 + 7 = 12\n\nTo solve it, first subtract 7 from both sides to isolate x^3:\n\nx^3 = 12 - 7\nx^3 = 5\n\nThen, take the cube root of both sides to solve for x:\n\nx = ∛5')

## 添加一个Attaching OpenAI tools（添加OpenAI工具）

In [28]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [29]:
model = ChatOpenAI(
  model="gpt-3.5-turbo-1106",
  ).bind(tools=tools)
model.invoke("What's the weather in SF, NYC and LA?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kprXENTuFgCyg4xiX9cjAW5v', 'function': {'arguments': '{"location": "San Francisco, CA", "unit": "celsius"}', 'name': 'get_current_weather'}, 'type': 'function'}, {'id': 'call_5m8eVu0TtuyBfGG93j5G6MRo', 'function': {'arguments': '{"location": "New York, NY", "unit": "celsius"}', 'name': 'get_current_weather'}, 'type': 'function'}, {'id': 'call_LJ0kddrxForxF7GUbNiIFfBq', 'function': {'arguments': '{"location": "Los Angeles, CA", "unit": "celsius"}', 'name': 'get_current_weather'}, 'type': 'function'}]})